# Module 4, Activity 2: Integrating Other Services into an Agent 

In the previous activity we saw how to use an LLM to call an API by creating a tool.  In this activity, we will see how we can make tools to call other services, such as querying databases.  We will demonstrate this in SQL, but remember that you can incorporate any AWS service into your GenAI pipeline by turning it into a tool and binding it to your LLM.

In [ ]:
import boto3
import sqlite3

from langchain.agents import AgentExecutor, create_tool_calling_agent, initialize_agent
from langchain.agents.agent_types import AgentType
from langchain_aws import ChatBedrock, ChatBedrockConverse
from langchain.prompts import ChatPromptTemplate
from langchain.tools import Tool, tool

In [ ]:
session = boto3.session.Session()
region = session.region_name
bedrock_runtime = boto3.client("bedrock-runtime", region_name='us-west-2')

## Create an in-memory SQL database

For this activity we are going to create a quick, in-memory SQLite database that we can query with a tool.  So let's just mock something up really quick.

In [ ]:
import sqlite3

def setup_sqlite():
    conn = sqlite3.connect(":memory:")
    cursor = conn.cursor()

    cursor.executescript("""
    DROP TABLE IF EXISTS orders;
    DROP TABLE IF EXISTS customers;

    CREATE TABLE customers (
        id INTEGER PRIMARY KEY,
        name TEXT,
        email TEXT
    );

    CREATE TABLE orders (
        id INTEGER PRIMARY KEY,
        customer_id INTEGER,
        amount REAL,
        date TEXT,
        FOREIGN KEY(customer_id) REFERENCES customers(id)
    );

    INSERT INTO customers (id, name, email) VALUES
    (1, 'Alice Smith', 'alice@example.com'),
    (2, 'Bob Johnson', 'bob@example.com'),
    (3, 'Carol Lee', 'carol@example.com'),
    (4, 'David Kim', 'david@example.com'),
    (5, 'Ella Brown', 'ella@example.com'),
    (6, 'Frank White', 'frank@example.com'),
    (7, 'Grace Green', 'grace@example.com'),
    (8, 'Henry Black', 'henry@example.com'),
    (9, 'Ivy Adams', 'ivy@example.com'),
    (10, 'Jack Nelson', 'jack@example.com');

    INSERT INTO orders (id, customer_id, amount, date) VALUES
    (1, 1, 125.50, '2024-01-01'),
    (2, 1, 75.00, '2024-02-14'),
    (3, 2, 300.00, '2024-02-20'),
    (4, 3, 50.25, '2024-03-15'),
    (5, 4, 220.00, '2024-03-30'),
    (6, 5, 180.75, '2024-04-01'),
    (7, 6, 90.00, '2024-04-12'),
    (8, 6, 130.00, '2024-04-15'),
    (9, 7, 55.55, '2024-04-20'),
    (10, 8, 200.00, '2024-04-22'),
    (11, 9, 300.00, '2024-04-25'),
    (12, 10, 150.00, '2024-04-27'),
    (13, 2, 100.00, '2024-04-28'),
    (14, 3, 400.00, '2024-04-29'),
    (15, 5, 250.00, '2024-04-30');
    """)

    conn.commit()
    return conn


## Creating the SQL tool

This code defines a function `make_sql_tool` that returns a LangChain-compatible tool for executing SQL `SELECT` queries against an in-memory SQLite database.  Inside `make_sql_tool`, a nested function `execute_sql_query` is decorated with `@tool` like before, which allows LangChain agents to invoke it as a callable tool.  The reason `execute_sql_query` is defined inside `make_sql_tool` is to take advantage of a Python closure — this design pattern allows the nested function to access the `conn` database connection passed into the outer function without including it as a parameter in the tool interface.  This keeps the tool signature compatible with LangChain’s expectation of a single input: `str` argument while still enabling safe, dynamic access to the SQLite connection.

In [ ]:
def make_sql_tool(conn: sqlite3.Connection):
    @tool
    def execute_sql_query(input: str) -> str:
        """
        Execute a SQL SELECT query on an in-memory SQLite database with 'customers' and 'orders' tables.
        Input should be a valid SQL SELECT statement.
        Do not use DROP, DELETE, or INSERT statements.
        """
        try:
            if not input.strip().lower().startswith("select"):
                return "Only SELECT statements are allowed."

            cursor = conn.cursor()
            cursor.execute(input)
            rows = cursor.fetchall()

            if not rows:
                return "No results."

            col_names = [description[0] for description in cursor.description]
            result = "\n".join([str(dict(zip(col_names, row))) for row in rows])
            return result
        except Exception as e:
            return f"SQL Error: {str(e)}"

    return execute_sql_query

## Creating the initial agent

We now create our connection to the SQLite database and use that to create the tool with our above function.  This tool is then passed into `initialize_agent` like we saw in the previous activity.

In [ ]:
conn = setup_sqlite()
sql_tool = make_sql_tool(conn)

tools = [sql_tool]

llm = ChatBedrockConverse(
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    temperature=0.0,
)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [ ]:
query = "Which customer has the highest total spend?"
response = agent.invoke(query)
print(response)

## Oops!

Well, that was interesting!  What did you notice?  

It turns out that the more information you can tell the LLM about the structured database, the better.  So let's do that again, but this time let's create a system prompt that provides the SQL schema to be queried.

In [ ]:
schema_description = """
You are a helpful assistant that answers questions about a SQLite database.

Database schema:
Table: customers
- id: INTEGER
- name: TEXT
- email: TEXT

Table: orders
- id: INTEGER
- customer_id: INTEGER
- amount: REAL
- date: TEXT
(customer_id is a foreign key to customers.id)

You may only execute SELECT queries.
Do not perform any INSERT, DELETE, or UPDATE operations.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", schema_description),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [ ]:
tools = [make_sql_tool(conn)]

llm = ChatBedrockConverse(
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    temperature=0.0,
)

agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
query = "Which customer has the highest total spend?"
response = agent_executor.invoke({"input": query})
print(response)

In [ ]:
query = "List the total amount spent by each customer, sorted from highest to lowest.  Provide your answer in natural language."
response = agent_executor.invoke({"input": query})
print(response["output"])

## Concluding thoughts

Tools allow you to supercharge your LLM-based applications by providing them with additional functionality.  You can create a tool for just about anything, including the ability to query SQL like we did here.  And then all you have to do is describe what you want the agent to do in natural language.  The LLM was then able to take that natural language query, translate it into SQL, run the query, and get the results back (and even translate them back into natural language if we so wished).  